In [ ]:
import torch
from config import load_config
from data_prep import create_data_loader
from model import load_model_for_inference
from train import test_model
from utils import plot_confusion_matrix, plot_precision_recall, plot_roc_curve

In [ ]:
# load config
config = load_config("configs/config.yaml")

# load test data
test_loader = create_data_loader(config["testpath"], 
                                config["test_label_col"],
                                config['tokenizer_model'],
                                config['max_length'],
                                config['batch_size'],
                                shuffle=False)

# setting device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load model
model = load_model_for_inference(config, device)

# test model
accuracy, f1, y_true, y_pred = test_model(model, test_loader, device)

In [ ]:
# Plot confusion matrix
plot_confusion_matrix(y_true, y_pred, classes=[str(i) for i in range(config['num_labels'])])

# Plot precision-recall curve
plot_precision_recall(y_true, y_pred)

# Plot ROC curve
plot_roc_curve(y_true, y_pred)